In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
base_path = '/kaggle/input/aptos2019/'
train_csv = os.path.join(base_path, 'train_1.csv')
val_csv = os.path.join(base_path, 'valid.csv')
train_dir = os.path.join(base_path, 'train_images/train_images')
val_dir = os.path.join(base_path, 'val_images/val_images')


In [ ]:

train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)


# This block calculates weights to counteract the class imbalance.
print("Calculating class weights...")
class_labels = np.unique(train_df['diagnosis'])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=class_labels,
    y=train_df['diagnosis'].values
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print(f"Calculated Weights: {weights_tensor}")



Calculating class weights...
Calculated Weights: tensor([0.4086, 1.9533, 0.7252, 3.8052, 2.5043])


In [ ]:
# 3. --- DEFINE ADVANCED AUGMENTATIONS ---
## --- UPDATE: MORE POWERFUL AUGMENTATIONS --- ##
# We've added more transformations like vertical flips, rotation, and color jitter.
print("Defining advanced data augmentations...")
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Validation transforms remain simple.
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])
## --- END OF UPDATE --- ##


Defining advanced data augmentations...


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
# 4. --- CREATE PYTORCH DATASET ---
# (No changes here, it's designed to work with the transforms)
class DRDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.iloc[index]['id_code'] + '.png'
        label = self.df.iloc[index]['diagnosis']
        img_path = os.path.join(self.image_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label


In [ ]:
# 5. --- CREATE DATALOADERS ---
# (No changes here)
train_dataset = DRDataset(train_df, train_dir, transform=train_transform)
val_dataset = DRDataset(val_df, val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
# 6. --- DEFINE HYBRID MODEL ---
# (No changes here)
# !pip install -q timm # Ensure timm is installed
class HybridModel(nn.Module):
    def __init__(self, num_classes=5):
        super(HybridModel, self).__init__()
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0)
        self.resnet = timm.create_model('resnet18', pretrained=True, num_classes=0)
        self.efficientnet_out = self.efficientnet.num_features
        self.resnet_out = self.resnet.num_features
        self.fusion = nn.Sequential(
            nn.Linear(self.efficientnet_out + self.resnet_out, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        feat1 = self.efficientnet(x)
        feat2 = self.resnet(x)
        combined = torch.cat((feat1, feat2), dim=1)
        out = self.fusion(combined)
        return out

In [ ]:
# 7. --- SET UP FOR TRAINING ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=5).to(device)

## --- UPDATE: LOSS FUNCTION AND OPTIMIZER --- ##
# We move the weights tensor to the correct device.
# We initialize the loss with both the `weight` and `label_smoothing` parameters.
weights_tensor = weights_tensor.to(device)
criterion = nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print(f"\nTraining on device: {device}")
print("Using CrossEntropyLoss with class weights and label smoothing.")
## --- END OF UPDATE --- ##



Training on device: cuda
Using CrossEntropyLoss with class weights and label smoothing.


In [ ]:
# 8. --- TRAINING AND VALIDATION LOOP ---
# (No changes to the loop logic)
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100 * correct / total
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")


Epoch 1/30 [Train]: 100%|██████████| 92/92 [02:36<00:00,  1.70s/it]


Train Loss: 1.5301 | Train Accuracy: 52.29%


Epoch 2/30 [Train]: 100%|██████████| 92/92 [02:32<00:00,  1.66s/it]


Train Loss: 1.2958 | Train Accuracy: 68.67%


Epoch 3/30 [Train]: 100%|██████████| 92/92 [02:33<00:00,  1.67s/it]


...

Train Loss: 0.7205 | Train Accuracy: 96.89%


Epoch 29/30 [Train]: 100%|██████████| 92/92 [02:32<00:00,  1.66s/it]


Train Loss: 0.7209 | Train Accuracy: 97.00%


Epoch 30/30 [Train]: 100%|██████████| 92/92 [02:34<00:00,  1.68s/it]

Train Loss: 0.7211 | Train Accuracy: 96.69%


In [ ]:

model.eval()
val_correct = 0
val_total = 0
with torch.no_grad():
    for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.2f}%\n")

print("Training complete.")

Epoch 30/30 [Val]: 100%|██████████| 12/12 [00:20<00:00,  1.73s/it]

Validation Accuracy: 83.06%

Training complete.


In [ ]:
# Add these to your other imports
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
model.eval()
    # Create empty lists to store predictions and true labels
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # Append batch predictions and labels to the lists
            # Move to CPU before appending
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Now calculate metrics using the collected predictions and labels
    print("\n--- Validation Metrics ---")

    # 1. Classification Report (Precision, Recall, F1-Score)
    # The classification_report gives you precision, recall, and f1-score per class.
    class_names = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']
    report = classification_report(all_labels, all_preds, target_names=class_names)
    print(report)

    # 2. Specificity Calculation
    # Specificity = True Negatives / (True Negatives + False Positives)
    cm = confusion_matrix(all_labels, all_preds)
    total_samples = cm.sum()
    print("Specificity per class:")
    for i, class_name in enumerate(class_names):
        tn = total_samples - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
        fp = np.sum(cm[:, i]) - cm[i, i]
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        print(f"  - {class_name}: {specificity:.4f}")

    # 3. Confusion Matrix Visualization
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix for Epoch {epoch+1}')
    plt.show()
    print("-" * 25 + "\n")
    ## --- END OF UPDATE --- ##

print("Training complete.")